# Croping notebook

This is an example notebook for cropping long light-sheet movies based on the sample of interest.

## Importing and defining system memory

Important to note that the AVAILABLE_SYSTEM_MEMORY should ideally be below 80% of the available RAM to prevent the process from being killed by the operating system.

In [ ]:
import luigi
import pandas as pd
import multiprocessing
from lstree.cropping.candidates import BatchedFindCropCandidateTask
from lstree.cropping.crop_gui import CropReviewer
from lstree.cropping.crop import CropMovieTask
import holoviews as hv
from holoviews import opts

opts.defaults(opts.Rectangles(selection_fill_color=None),
              opts.Layout(sizing_mode='fixed'))


AVAILABLE_SYSTEM_MEMORY = 64000 # in MB 

luigi.configuration.get_config().set('resources', 'memory', str(AVAILABLE_SYSTEM_MEMORY))
luigi.configuration.get_config().set('resources', 'pool_workers', str(multiprocessing.cpu_count()))
luigi.configuration.get_config().set('core', 'workers', str(multiprocessing.cpu_count()))


hv.extension('bokeh', width=100)

# Define parameters

In [ ]:
spacing = (2, 0.173, 0.173) # voxel size
ref_channel = 'Channel1' # channel used to find objects

max_clip_factor = 2 # maximum intensity clipping at: factor x mean(image)
sigma = 2 # blurring size prior thresholding
threshold_method = 'otsu' # thresholding method. one of ['otsu', 'li', 'yen', 'triangle', 'minimum'] or a callable

# Use larger step to test  segmentation parameters (e.g. step=50)
# use all timepoints for actual cropping step=1
timepoint_step = 1

input_dir = 'PATH_TO_RAW_IMAGES'
output_dir = 'CROP_OUTPUT_FOLDER'

file_pattern = '{channel}-T{time:04d}.{ext}' 
index = ['channel', 'time']

# Find objects in stack
existing files are skipped

In [ ]:
batch_crop_task = BatchedFindCropCandidateTask(datadir=input_dir,
                                               out_directory=output_dir,
                                               pattern=file_pattern,
                                               index=index,
                                               timepoint_step=timepoint_step,
                                               ref_channel=ref_channel,
                                               spacing=spacing,
                                               sigma=sigma,
                                               max_clip_factor=max_clip_factor,
                                               threshold_method=threshold_method,
                                               memory=15000) # ~memory (MB) required to process a single stack

luigi.build([batch_crop_task], local_scheduler=True, log_level='INFO')

# Review cropping, starting from the last timepoint

- **white** detected objects
- **orange** selected object
- **red** movie cropping box, centered on current object  
  
  
1. click on the object you want to crop, backtracking will update automatically
1. In case of error, click on a different object to update the backtracking from the current timepoint
1. Occasionally, If there is no suitable object detected, the current box can be edited manually with the sliders
1. If that happens often, try to rerun the detection with different segmentation parameters
1. Save the list of bounding boxes as "object 0"
1. restart by selecting the next object from the last frame


In [ ]:
df = pd.read_hdf(batch_crop_task.output().path, key='dc')
df_mips = pd.read_hdf(batch_crop_task.output().path, key='MIPs')
dfb = pd.read_hdf(batch_crop_task.output().path, key='boxes')
    
cr = CropReviewer(spacing=spacing,
                  df_mips=df_mips,
                  dfb=dfb,
                  outdir=output_dir,
                  frame_width=600)

cr.panel().servable()

# Crop images

In [ ]:
crop_movie_task = CropMovieTask(out_directory=output_dir,
                                data_collection=batch_crop_task.output().path,
                                n_threads=multiprocessing.cpu_count(),
                                frame_memory=2000, # ~memory (MB) required to process a single stack
                                objects_csv=['crop_roi_0.csv'])

luigi.build([crop_movie_task], local_scheduler=True, log_level='DEBUG')